# 🎮 디시인사이드 크롤링 + KoBERT 감성분석

## 학습 목표
1. **디시인사이드 갤러리** 게시글 및 댓글 크롤링
2. **KoBERT 기반 감성분석** 수행
3. **혐오표현 탐지** 및 분석

---

## 📚 목차
1. [환경 설정](#1-환경-설정)
2. [갤러리 설정](#2-갤러리-설정)
3. [게시글 목록 수집](#3-게시글-목록-수집)
4. [댓글 수집](#4-댓글-수집)
5. [텍스트 전처리](#5-텍스트-전처리)
6. [KoBERT 감성분석](#6-kobert-감성분석)
7. [혐오표현 탐지](#7-혐오표현-탐지)
8. [종합 분석](#8-종합-분석)

---

## ⚠️ 주의사항
- 교육/연구 목적으로만 사용하세요
- 과도한 크롤링은 서버에 부담을 줍니다
- 개인정보 보호에 유의하세요

---
# 1. 환경 설정

In [ ]:
# ============================================
# 1-1. 라이브러리 설치
# ============================================

!pip install requests beautifulsoup4 lxml -q
!pip install konlpy -q
!pip install pandas numpy matplotlib seaborn wordcloud -q
!pip install transformers torch sentencepiece -q

# Selenium (Colab용)
!pip install selenium -q
!apt-get update -qq
!apt-get install -y chromium-chromedriver -qq

print("✅ 설치 완료!")

In [ ]:
# ============================================
# 1-2. Colab 환경 설정
# ============================================

import os
import sys
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

def setup_colab():
    print("🔄 Java 설치 중...")
    os.system('apt-get install -y openjdk-11-jdk-headless -qq')
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    
    print("🔄 한글 폰트 설치 중...")
    os.system('apt-get install -y fonts-nanum -qq')
    fm._load_fontmanager()
    
    plt.rcParams['font.family'] = 'NanumBarunGothic'
    plt.rcParams['axes.unicode_minus'] = False
    
    # Selenium 설정
    sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
    
    print("✅ 완료!")

try:
    import google.colab
    setup_colab()
    IN_COLAB = True
except:
    plt.rcParams['font.family'] = 'Malgun Gothic'
    IN_COLAB = False
    print("로컬 환경입니다.")

In [ ]:
# ============================================
# 1-3. 라이브러리 임포트
# ============================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import json
import time
from collections import Counter
from datetime import datetime
from urllib.parse import quote, urlencode
import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Okt
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

print("✅ 임포트 완료!")
print(f"🔧 GPU: {torch.cuda.is_available()}")

---
# 2. 갤러리 설정

## 📌 디시인사이드 구조
```
갤러리 (Gallery)
    ↓
게시글 목록 (List)
    ↓
게시글 상세 + 댓글 (View + Comments)
```

## 🔗 주요 갤러리 ID 예시
| 갤러리 | ID |
|--------|----|
| 주식 | `stock` |
| 비트코인 | `bitcoins` |
| 야구 | `baseball_new10` |
| 국내야구 | `kbaseball` |
| 축구 | `football_new6` |
| 정치 | `politics` |
| 인터넷방송 | `ib` |

In [ ]:
# ============================================
# 2-1. 수집 설정
# ============================================

# ⬇️⬇️⬇️ 여기를 수정하세요! ⬇️⬇️⬇️

# 갤러리 ID (URL에서 확인: gall.dcinside.com/board/lists/?id=여기)
GALLERY_ID = "politics"  # 정치 갤러리

# 또는 마이너 갤러리
IS_MINOR = False  # 마이너 갤러리면 True

# 검색 키워드 (선택사항, 빈 문자열이면 전체 글)
KEYWORD = "이태원"  # 검색어

# 수집 설정
MAX_PAGES = 5           # 수집할 페이지 수
MAX_POSTS = 30          # 최대 게시글 수
MAX_COMMENTS = 50       # 게시글당 최대 댓글 수

# ⬆️⬆️⬆️ 여기를 수정하세요! ⬆️⬆️⬆️

print(f"🎮 갤러리: {GALLERY_ID}")
print(f"🔍 키워드: {KEYWORD if KEYWORD else '(전체)'}")
print(f"📄 페이지: {MAX_PAGES}개")
print(f"📝 게시글: 최대 {MAX_POSTS}개")
print(f"💬 댓글: 게시글당 {MAX_COMMENTS}개")

In [ ]:
# ============================================
# 2-2. 요청 헤더 설정
# ============================================

# 디시인사이드는 User-Agent와 Referer 체크를 함
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://gall.dcinside.com/',
}

# 세션 생성 (쿠키 유지)
session = requests.Session()
session.headers.update(HEADERS)

print("✅ 세션 설정 완료!")

---
# 3. 게시글 목록 수집

In [ ]:
# ============================================
# 3-1. 갤러리 URL 생성
# ============================================

def get_gallery_url(gallery_id, page=1, keyword="", is_minor=False):
    """
    디시인사이드 갤러리 URL 생성
    """
    if is_minor:
        base = "https://gall.dcinside.com/mgallery/board/lists/"
    else:
        base = "https://gall.dcinside.com/board/lists/"
    
    params = {
        'id': gallery_id,
        'page': page,
    }
    
    if keyword:
        params['s_type'] = 'search_subject_memo'  # 제목+내용 검색
        params['s_keyword'] = keyword
    
    url = base + "?" + urlencode(params)
    return url

# 테스트
test_url = get_gallery_url(GALLERY_ID, 1, KEYWORD, IS_MINOR)
print(f"🔗 URL: {test_url}")

In [ ]:
# ============================================
# 3-2. 게시글 목록 크롤링 함수
# ============================================

def get_post_list(gallery_id, page=1, keyword="", is_minor=False):
    """
    갤러리 페이지에서 게시글 목록 수집
    
    Returns:
    --------
    list : 게시글 정보 리스트
    """
    
    posts = []
    url = get_gallery_url(gallery_id, page, keyword, is_minor)
    
    try:
        response = session.get(url, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'lxml')
        
        # 게시글 목록 찾기
        post_table = soup.select('tr.ub-content.us-post')
        
        for row in post_table:
            try:
                # 글번호
                num_elem = row.select_one('td.gall_num')
                post_num = num_elem.get_text(strip=True) if num_elem else ""
                
                # 공지, 설문 등 제외
                if not post_num.isdigit():
                    continue
                
                # 제목
                title_elem = row.select_one('td.gall_tit a:first-child')
                if not title_elem:
                    continue
                title = title_elem.get_text(strip=True)
                href = title_elem.get('href', '')
                
                # 댓글 수
                reply_elem = row.select_one('td.gall_tit a.reply_numbox')
                reply_count = 0
                if reply_elem:
                    reply_text = reply_elem.get_text(strip=True)
                    reply_match = re.search(r'\d+', reply_text)
                    if reply_match:
                        reply_count = int(reply_match.group())
                
                # 작성자
                writer_elem = row.select_one('td.gall_writer')
                writer = writer_elem.get_text(strip=True) if writer_elem else ""
                
                # 조회수
                count_elem = row.select_one('td.gall_count')
                view_count = count_elem.get_text(strip=True) if count_elem else "0"
                
                # 추천수
                recommend_elem = row.select_one('td.gall_recommend')
                recommend = recommend_elem.get_text(strip=True) if recommend_elem else "0"
                
                # 날짜
                date_elem = row.select_one('td.gall_date')
                date = date_elem.get('title', '') if date_elem else ""
                
                posts.append({
                    'num': post_num,
                    'title': title,
                    'url': 'https://gall.dcinside.com' + href if href.startswith('/') else href,
                    'writer': writer,
                    'view_count': view_count,
                    'recommend': recommend,
                    'reply_count': reply_count,
                    'date': date,
                })
                
            except Exception as e:
                continue
        
    except Exception as e:
        print(f"   ⚠️ 오류: {e}")
    
    return posts

In [ ]:
# ============================================
# 3-3. 여러 페이지 게시글 수집
# ============================================

all_posts = []

print(f"🔄 '{GALLERY_ID}' 갤러리 게시글 수집 중...")
if KEYWORD:
    print(f"   검색어: {KEYWORD}")
print()

for page in range(1, MAX_PAGES + 1):
    posts = get_post_list(GALLERY_ID, page, KEYWORD, IS_MINOR)
    
    if not posts:
        print(f"   페이지 {page}: 게시글 없음")
        break
    
    all_posts.extend(posts)
    print(f"   페이지 {page}: {len(posts)}개 수집 (총 {len(all_posts)}개)")
    
    if len(all_posts) >= MAX_POSTS:
        break
    
    time.sleep(0.5)  # 서버 부담 방지

# 최대 개수 제한
all_posts = all_posts[:MAX_POSTS]

print(f"\n✅ 총 {len(all_posts)}개 게시글 수집 완료!")

In [ ]:
# ============================================
# 3-4. 수집된 게시글 확인
# ============================================

if all_posts:
    print("\n📋 수집된 게시글:")
    print("="*70)
    
    for i, post in enumerate(all_posts[:10], 1):
        title = post['title'][:40] + "..." if len(post['title']) > 40 else post['title']
        print(f"{i}. {title}")
        print(f"   👤 {post['writer']} | 👁️ {post['view_count']} | 💬 {post['reply_count']} | 👍 {post['recommend']}")
else:
    print("❌ 수집된 게시글이 없습니다.")

---
# 4. 댓글 수집

In [ ]:
# ============================================
# 4-1. 게시글 상세 및 댓글 수집 함수
# ============================================

def get_post_comments(post_url, gallery_id, post_num, is_minor=False, max_comments=50):
    """
    게시글의 본문과 댓글 수집
    
    디시인사이드 댓글은 AJAX로 로드됨
    """
    
    result = {
        'content': '',
        'comments': []
    }
    
    try:
        # 1. 게시글 페이지 접근 (e_s_n_o 값 획득)
        response = session.get(post_url, timeout=15)
        soup = BeautifulSoup(response.text, 'lxml')
        
        # 본문 추출
        content_elem = soup.select_one('div.write_div')
        if content_elem:
            result['content'] = content_elem.get_text(strip=True)
        
        # e_s_n_o 값 찾기 (댓글 API에 필요)
        e_s_n_o = ""
        scripts = soup.find_all('script')
        for script in scripts:
            if script.string and 'e_s_n_o' in str(script.string):
                match = re.search(r"e_s_n_o\s*=\s*['\"]([^'\"]+)['\"]", str(script.string))
                if match:
                    e_s_n_o = match.group(1)
                    break
        
        # 2. 댓글 API 호출
        if is_minor:
            comment_url = "https://gall.dcinside.com/mgallery/board/comment/"
        else:
            comment_url = "https://gall.dcinside.com/board/comment/"
        
        comment_headers = HEADERS.copy()
        comment_headers['X-Requested-With'] = 'XMLHttpRequest'
        comment_headers['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
        comment_headers['Referer'] = post_url
        
        # 댓글 데이터 요청
        data = {
            'id': gallery_id,
            'no': post_num,
            'cmt_id': gallery_id,
            'cmt_no': post_num,
            'e_s_n_o': e_s_n_o,
            'comment_page': 1,
            'sort': '',
            '_GALLTYPE_': 'M' if is_minor else 'G',
        }
        
        comment_response = session.post(comment_url, data=data, headers=comment_headers, timeout=15)
        
        if comment_response.status_code == 200:
            try:
                comment_data = comment_response.json()
                comments_html = comment_data.get('comments', '')
                
                if comments_html:
                    comment_soup = BeautifulSoup(comments_html, 'lxml')
                    comment_items = comment_soup.select('li.ub-content')
                    
                    for item in comment_items[:max_comments]:
                        # 댓글 내용
                        text_elem = item.select_one('p.usertxt')
                        if not text_elem:
                            continue
                        text = text_elem.get_text(strip=True)
                        
                        # 작성자
                        writer_elem = item.select_one('span.nickname')
                        writer = writer_elem.get_text(strip=True) if writer_elem else ""
                        
                        # 날짜
                        date_elem = item.select_one('span.date_time')
                        date = date_elem.get_text(strip=True) if date_elem else ""
                        
                        result['comments'].append({
                            'text': text,
                            'writer': writer,
                            'date': date,
                        })
                        
            except:
                pass
        
    except Exception as e:
        pass
    
    return result

In [ ]:
# ============================================
# 4-2. 모든 게시글에서 댓글 수집
# ============================================

all_comments = []
all_contents = []

if all_posts:
    print(f"🔄 {len(all_posts)}개 게시글에서 댓글 수집 중...\n")
    
    for i, post in enumerate(all_posts, 1):
        title_short = post['title'][:35] + "..." if len(post['title']) > 35 else post['title']
        print(f"[{i}/{len(all_posts)}] {title_short}")
        
        # 댓글 수집
        result = get_post_comments(
            post['url'], 
            GALLERY_ID, 
            post['num'], 
            IS_MINOR, 
            MAX_COMMENTS
        )
        
        # 본문 저장
        if result['content']:
            all_contents.append({
                'type': 'post',
                'text': result['content'],
                'title': post['title'],
                'writer': post['writer'],
                'url': post['url'],
            })
        
        # 댓글 저장
        for comment in result['comments']:
            comment['type'] = 'comment'
            comment['post_title'] = post['title']
            comment['post_url'] = post['url']
            all_comments.append(comment)
        
        print(f"      ✅ 댓글 {len(result['comments'])}개")
        
        time.sleep(0.5)  # 서버 부담 방지
    
    print(f"\n{'='*50}")
    print(f"✅ 총 {len(all_comments)}개 댓글 수집 완료!")
    print(f"✅ 총 {len(all_contents)}개 본문 수집 완료!")

In [ ]:
# ============================================
# 4-3. 샘플 데이터 (수집 실패 시)
# ============================================

def create_sample_dc_data():
    """디시인사이드 스타일 샘플 데이터"""
    samples = [
        # 일반 댓글
        {"text": "ㄹㅇ 인정 이건 맞는 말이네", "likes": 45},
        {"text": "아니 이게 뭔 소리야 ㅋㅋㅋㅋ", "likes": 32},
        {"text": "와 진짜 역대급이다", "likes": 28},
        {"text": "이거 실화냐?", "likes": 25},
        {"text": "ㅇㅈ 완전 공감", "likes": 22},
        
        # 비판적 댓글
        {"text": "이게 나라냐 진짜", "likes": 156},
        {"text": "책임지는 놈이 없네 ㅋㅋ", "likes": 134},
        {"text": "아직도 이 모양이냐", "likes": 98},
        {"text": "정신 차려라 진짜", "likes": 87},
        {"text": "답이 없다 답이 없어", "likes": 76},
        
        # 정치적 댓글
        {"text": "좌파들 특징 맨날 저러면서 남탓함", "likes": 65},
        {"text": "수꼴들은 뭘 해도 찬양이네", "likes": 54},
        {"text": "586 꼰대들이 문제야", "likes": 48},
        {"text": "틀딱들 투표 잘못해서 이 지경", "likes": 42},
        {"text": "MZ들은 맨날 불평만", "likes": 38},
        
        # 혐오 표현
        {"text": "그쪽 애들 항상 저러더라", "likes": 35},
        {"text": "역시 한심하네 수준이", "likes": 32},
        {"text": "멍청한 것들 ㅋㅋ", "likes": 28},
        {"text": "니들이 뭘 알아", "likes": 25},
        
        # 중립/정보성
        {"text": "팩트체크 좀 하고 말해라", "likes": 67},
        {"text": "소스 어디냐", "likes": 45},
        {"text": "일단 기다려보자", "likes": 34},
        {"text": "양쪽 다 문제 있음", "likes": 29},
        {"text": "객관적으로 보면 이게 맞음", "likes": 25},
    ]
    
    df = pd.DataFrame(samples)
    df['type'] = 'comment'
    df['post_title'] = f'{KEYWORD} 관련 글'
    df['post_url'] = f'https://gall.dcinside.com/board/view/?id={GALLERY_ID}'
    df['writer'] = [f'ㅇㅇ({i})' for i in range(len(df))]
    df['date'] = '2024.10.29'
    
    return df

# 데이터프레임 생성
if all_comments:
    df_comments = pd.DataFrame(all_comments)
    if 'likes' not in df_comments.columns:
        df_comments['likes'] = 0
else:
    print("\n⚠️ 댓글 수집 실패. 샘플 데이터 사용.")
    df_comments = create_sample_dc_data()

print(f"\n📊 총 {len(df_comments)}개 댓글")
df_comments.head()

---
# 5. 텍스트 전처리

In [ ]:
# ============================================
# 5-1. 디시 특화 전처리
# ============================================

okt = Okt()

# 디시인사이드 특화 불용어
DC_STOPWORDS = {
    '것', '수', '등', '때', '중', '좀', '더', '네', '거', '뭐', '제', '저', '게',
    '이', '그', '저', '나', '너', '우리', '저희', '하다', '되다', '있다', '없다',
    '진짜', '정말', '너무', '매우', '아주', '완전', '그냥', '바로', '다시', '또',
    # 디시 특화
    'ㅋㅋ', 'ㅋㅋㅋ', 'ㅋㅋㅋㅋ', 'ㅎㅎ', 'ㅎㅎㅎ', 'ㄹㅇ', 'ㅇㅈ', 'ㅇㅇ',
    'ㄱㄱ', 'ㄴㄴ', 'ㅇㅋ', 'ㅅㄱ', '갤러리', '글', '댓글', '추천', '비추',
}

def clean_dc_text(text):
    """디시인사이드 텍스트 정규화"""
    if pd.isna(text): return ""
    text = str(text)
    
    # 디시콘(이모티콘) 제거
    text = re.sub(r'\[.*?\]', '', text)
    
    # URL 제거
    text = re.sub(r'http\S+', '', text)
    
    # 반복 문자 정규화 (ㅋㅋㅋㅋㅋ → ㅋㅋ)
    text = re.sub(r'(ㅋ|ㅎ|ㅠ|ㅜ|ㄷ|ㅂ){3,}', r'\1\1', text)
    
    # 특수문자 제거 (한글, 영문, 숫자만 유지)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    
    # 연속 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def extract_dc_keywords(text):
    """디시 댓글에서 키워드 추출"""
    if not text: return []
    nouns = okt.nouns(str(text))
    return [w for w in nouns if len(w) >= 2 and w not in DC_STOPWORDS]

# 적용
print("🔄 전처리 중...")
df_comments['cleaned_text'] = df_comments['text'].apply(clean_dc_text)
df_comments['keywords'] = df_comments['cleaned_text'].apply(extract_dc_keywords)
df_comments['text_length'] = df_comments['cleaned_text'].apply(len)

# 빈 텍스트 제거
df_comments = df_comments[df_comments['text_length'] > 0].reset_index(drop=True)

print(f"✅ 완료! ({len(df_comments)}개)")

In [ ]:
# ============================================
# 5-2. 워드클라우드
# ============================================

all_keywords = [w for kws in df_comments['keywords'] for w in kws]
word_freq = Counter(all_keywords)

if word_freq:
    font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' if IN_COLAB else 'C:/Windows/Fonts/malgun.ttf'
    
    wc = WordCloud(
        font_path=font_path, 
        width=900, height=400,
        background_color='white', 
        colormap='viridis',
        max_words=80
    ).generate_from_frequencies(word_freq)
    
    plt.figure(figsize=(14, 6))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'📊 디시인사이드 "{GALLERY_ID}" 갤러리 키워드', fontsize=14, fontweight='bold')
    plt.show()

print("\n🔑 TOP 15 키워드:")
for w, c in word_freq.most_common(15):
    print(f"   {w}: {c}회")

---
# 6. KoBERT 감성분석

In [ ]:
# ============================================
# 6-1. KoBERT 모델 로드
# ============================================

print("🔄 KoBERT 모델 로딩...")

MODEL_NAME = "snunlp/KR-FinBert-SC"

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    
    sentiment_analyzer = pipeline(
        "sentiment-analysis",
        model=model, tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1,
        max_length=512, truncation=True
    )
    
    print(f"✅ 완료! ({MODEL_NAME})")
    MODEL_LOADED = True
except Exception as e:
    print(f"❌ 실패: {e}")
    MODEL_LOADED = False

In [ ]:
# ============================================
# 6-2. 감성분석 함수 (디시 특화)
# ============================================

def analyze_sentiment_dc(text):
    """KoBERT 감성분석 + 디시 특화 규칙"""
    
    if not text or len(str(text).strip()) < 2:
        return 'neutral', 0.5, 0.0
    
    if MODEL_LOADED:
        try:
            result = sentiment_analyzer(str(text)[:500])[0]
            label = result['label'].lower()
            score = result['score']
            
            if 'positive' in label or 'pos' in label:
                return 'positive', score, score
            elif 'negative' in label or 'neg' in label:
                return 'negative', score, -score
            return 'neutral', score, 0.0
        except:
            pass
    
    # 규칙 기반 백업 (디시 특화)
    text_lower = str(text).lower()
    
    pos_patterns = ['인정', '좋', '굿', 'ㅇㅈ', '공감', '맞는말', '팩트', '레전드']
    neg_patterns = ['아니', '싫', '별로', '노답', '답없', '한심', '멍청', '병신']
    
    pos_count = sum(1 for p in pos_patterns if p in text_lower)
    neg_count = sum(1 for p in neg_patterns if p in text_lower)
    
    if pos_count > neg_count:
        return 'positive', 0.7, 0.5
    elif neg_count > pos_count:
        return 'negative', 0.7, -0.5
    return 'neutral', 0.5, 0.0

In [ ]:
# ============================================
# 6-3. 전체 감성분석 수행
# ============================================

print("🔄 감성분석 진행 중...")

results = [analyze_sentiment_dc(text) for text in df_comments['cleaned_text']]

df_comments['sentiment'] = [r[0] for r in results]
df_comments['sentiment_confidence'] = [r[1] for r in results]
df_comments['sentiment_score'] = [r[2] for r in results]

print("\n✅ 완료!")
print("\n📊 감성 분포:")
print(df_comments['sentiment'].value_counts())

In [ ]:
# ============================================
# 6-4. 감성분석 시각화
# ============================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

colors = {'positive': '#27ae60', 'neutral': '#95a5a6', 'negative': '#e74c3c'}
sentiment_counts = df_comments['sentiment'].value_counts()

# 감성 분포
axes[0].pie(sentiment_counts.values, 
            labels=['긍정', '중립', '부정'][:len(sentiment_counts)],
            autopct='%1.1f%%', 
            colors=[colors.get(s, '#95a5a6') for s in sentiment_counts.index])
axes[0].set_title('🤖 KoBERT 감성 분포', fontweight='bold')

# 신뢰도 분포
axes[1].hist(df_comments['sentiment_confidence'], bins=20, color='steelblue', alpha=0.7)
axes[1].axvline(df_comments['sentiment_confidence'].mean(), color='red', linestyle='--')
axes[1].set_xlabel('신뢰도')
axes[1].set_title('📊 신뢰도 분포', fontweight='bold')

# 감성별 텍스트 길이
df_comments.boxplot(column='text_length', by='sentiment', ax=axes[2])
axes[2].set_xlabel('감성')
axes[2].set_ylabel('텍스트 길이')
axes[2].set_title('📈 감성별 텍스트 길이', fontweight='bold')
plt.suptitle('')

plt.tight_layout()
plt.show()

---
# 7. 혐오표현 탐지

In [ ]:
# ============================================
# 7-1. 디시인사이드 특화 혐오표현 패턴
# ============================================

DC_HATE_PATTERNS = {
    '일반화': ['항상', '맨날', '늘', '전부', '걔네', '쟤네', '그쪽', '니들', '무조건', '역시나'],
    '비하': ['한심', '멍청', '무식', '바보', '병신', '븅신', '미친', '또라이', '찐따', '씹', '좆', '꼴통', '수준'],
    '정치혐오': ['좌파', '우파', '좌좀', '빨갱이', '수꼴', '꼴통', '문빠', '윤빠', '민주당충', '국힘충', '찍충'],
    '세대혐오': ['틀딱', '꼰대', '586', '젊은것', '늙은이', '요즘것들', 'MZ충', '라떼', '노인네'],
    '성별혐오': ['페미', '한남', '한녀', '김치녀', '맘충', '냄저', '보빨', '재기'],
    '지역혐오': ['홍어', '쪽바리', '짱깨', '전라도', '경상도'],
    '외모비하': ['못생', '돼지', '뚱땡이', '대머리', '키작'],
}

def detect_dc_hate(text):
    """디시 특화 혐오표현 탐지"""
    if not text: return [], [], 0
    text = str(text).lower()
    found_types, found_words = [], []
    
    for htype, patterns in DC_HATE_PATTERNS.items():
        for p in patterns:
            if p.lower() in text:
                if htype not in found_types:
                    found_types.append(htype)
                found_words.append(p)
    
    return found_types, found_words, len(found_types)*2 + len(found_words)

# 적용
print("🔄 혐오표현 탐지 중...")

hate_results = [detect_dc_hate(t) for t in df_comments['cleaned_text']]
df_comments['hate_types'] = [r[0] for r in hate_results]
df_comments['hate_words'] = [r[1] for r in hate_results]
df_comments['hate_score'] = [r[2] for r in hate_results]
df_comments['has_hate'] = df_comments['hate_types'].apply(lambda x: len(x) > 0)

hate_ratio = df_comments['has_hate'].mean() * 100

print(f"\n✅ 완료!")
print(f"   혐오 포함: {df_comments['has_hate'].sum()}개 ({hate_ratio:.1f}%)")

In [ ]:
# ============================================
# 7-2. 혐오표현 유형별 분석
# ============================================

all_hate_types = [t for types in df_comments['hate_types'] for t in types]
type_freq = Counter(all_hate_types)

print("\n📊 혐오표현 유형별 빈도:")
print("="*50)
for htype, count in type_freq.most_common():
    pct = count / len(df_comments) * 100
    bar = '█' * min(count, 15)
    print(f"   {htype:12} | {count:3}개 ({pct:4.1f}%) | {bar}")

In [ ]:
# ============================================
# 7-3. 혐오표현 시각화
# ============================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 혐오 비율
axes[0].pie([df_comments['has_hate'].sum(), (~df_comments['has_hate']).sum()],
            labels=['혐오 포함', '정상'], autopct='%1.1f%%',
            colors=['#e74c3c', '#3498db'], explode=[0.05, 0])
axes[0].set_title(f'⚠️ 혐오표현 비율 ({hate_ratio:.1f}%)', fontweight='bold')

# 유형별 분포
if type_freq:
    types, counts = zip(*type_freq.most_common())
    axes[1].barh(types, counts, color=plt.cm.Reds(np.linspace(0.3, 0.9, len(types))))
    axes[1].set_xlabel('빈도')
    axes[1].invert_yaxis()
axes[1].set_title('🚫 혐오표현 유형', fontweight='bold')

# 감성 vs 혐오
cross = pd.crosstab(df_comments['sentiment'], df_comments['has_hate'])
cross.columns = ['정상', '혐오']
cross.plot(kind='bar', ax=axes[2], color=['#3498db', '#e74c3c'])
axes[2].set_xlabel('감성')
axes[2].set_title('📊 감성별 혐오표현', fontweight='bold')
axes[2].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 7-4. 혐오표현 포함 댓글 예시
# ============================================

hate_df = df_comments[df_comments['has_hate']].sort_values('hate_score', ascending=False)

if len(hate_df) > 0:
    print("\n⚠️ 혐오표현 포함 댓글 (점수 높은 순):")
    print("="*70)
    
    for i, (_, row) in enumerate(hate_df.head(8).iterrows(), 1):
        text = row['text'][:60] + "..." if len(row['text']) > 60 else row['text']
        print(f"\n{i}. [점수:{row['hate_score']}] [유형: {', '.join(row['hate_types'])}]")
        print(f"   {text}")

---
# 8. 종합 분석

In [ ]:
# ============================================
# 8-1. 종합 대시보드
# ============================================

fig = plt.figure(figsize=(16, 10))

# 1. 감성 분포
ax1 = fig.add_subplot(2, 3, 1)
sentiment_counts = df_comments['sentiment'].value_counts()
ax1.pie(sentiment_counts.values, labels=['긍정', '중립', '부정'][:len(sentiment_counts)],
        autopct='%1.1f%%', colors=[colors.get(s, '#95a5a6') for s in sentiment_counts.index])
ax1.set_title('🤖 KoBERT 감성 분포', fontweight='bold')

# 2. 혐오 비율
ax2 = fig.add_subplot(2, 3, 2)
ax2.pie([df_comments['has_hate'].sum(), (~df_comments['has_hate']).sum()],
        labels=['혐오', '정상'], autopct='%1.1f%%', colors=['#e74c3c', '#3498db'])
ax2.set_title(f'⚠️ 혐오표현 비율', fontweight='bold')

# 3. 키워드 TOP 10
ax3 = fig.add_subplot(2, 3, 3)
top10 = word_freq.most_common(10)
if top10:
    words, freqs = zip(*top10)
    ax3.barh(words, freqs, color='teal')
    ax3.invert_yaxis()
ax3.set_title('🔑 키워드 TOP 10', fontweight='bold')

# 4. 혐오 유형
ax4 = fig.add_subplot(2, 3, 4)
if type_freq:
    types, counts = zip(*type_freq.most_common())
    ax4.barh(types, counts, color=plt.cm.Reds(np.linspace(0.3, 0.9, len(types))))
    ax4.invert_yaxis()
ax4.set_title('🚫 혐오표현 유형', fontweight='bold')

# 5. 텍스트 길이 분포
ax5 = fig.add_subplot(2, 3, 5)
ax5.hist(df_comments['text_length'], bins=30, color='purple', alpha=0.7)
ax5.set_xlabel('텍스트 길이')
ax5.set_title('📏 텍스트 길이 분포', fontweight='bold')

# 6. 감성 점수 분포
ax6 = fig.add_subplot(2, 3, 6)
scatter_c = df_comments['has_hate'].map({True: '#e74c3c', False: '#3498db'})
ax6.scatter(df_comments['sentiment_score'], df_comments['text_length'], c=scatter_c, alpha=0.5, s=20)
ax6.set_xlabel('감성 점수')
ax6.set_ylabel('텍스트 길이')
ax6.set_title('📈 감성 vs 길이 (빨강=혐오)', fontweight='bold')

plt.suptitle(f'📊 디시인사이드 "{GALLERY_ID}" 갤러리 분석 결과', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 8-2. 분석 요약 리포트
# ============================================

print("\n" + "="*60)
print(f"📋 디시인사이드 '{GALLERY_ID}' 갤러리 분석 결과")
print("="*60)

print(f"\n[1] 데이터 개요")
print(f"   - 갤러리: {GALLERY_ID}")
print(f"   - 키워드: {KEYWORD if KEYWORD else '(전체)'}")
print(f"   - 게시글 수: {len(all_posts) if all_posts else 'N/A'}개")
print(f"   - 댓글 수: {len(df_comments):,}개")

print(f"\n[2] KoBERT 감성분석")
print(f"   - 모델: {MODEL_NAME if MODEL_LOADED else '규칙 기반'}")
for sent, cnt in df_comments['sentiment'].value_counts().items():
    print(f"   - {sent}: {cnt}개 ({cnt/len(df_comments)*100:.1f}%)")
print(f"   - 평균 신뢰도: {df_comments['sentiment_confidence'].mean():.3f}")

print(f"\n[3] 혐오표현 분석")
print(f"   - 혐오 포함: {df_comments['has_hate'].sum()}개 ({hate_ratio:.1f}%)")
if type_freq:
    print(f"   - 주요 유형: {', '.join([f'{t}({c})' for t, c in type_freq.most_common(3)])}")

print(f"\n[4] 주요 키워드")
print(f"   - {', '.join([w for w, c in word_freq.most_common(10)])}")

print("\n" + "="*60)

In [ ]:
# ============================================
# 8-3. 결과 저장
# ============================================

output_cols = ['text', 'cleaned_text', 'writer', 'sentiment', 'sentiment_confidence',
               'has_hate', 'hate_types', 'hate_words', 'hate_score']

df_output = df_comments[output_cols].copy()
df_output['hate_types'] = df_output['hate_types'].apply(lambda x: ', '.join(x) if x else '')
df_output['hate_words'] = df_output['hate_words'].apply(lambda x: ', '.join(x) if x else '')

filename = f"dcinside_{GALLERY_ID}_analysis.csv"
df_output.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"✅ '{filename}' 저장 완료!")

try:
    from google.colab import files
    files.download(filename)
except:
    print("로컬: 현재 폴더에서 확인하세요.")

---
# 🎓 학습 정리

## 디시인사이드 크롤링 구조
```
갤러리 ID 설정 → 게시글 목록 수집 → 댓글 API 호출 → 댓글 수집
```

## 주요 URL
- 일반 갤러리: `https://gall.dcinside.com/board/lists/?id=갤러리ID`
- 마이너 갤러리: `https://gall.dcinside.com/mgallery/board/lists/?id=갤러리ID`
- 댓글 API: `https://gall.dcinside.com/board/comment/`

## 디시 특화 전처리
- 반복 문자 정규화 (ㅋㅋㅋㅋ → ㅋㅋ)
- 디시콘 제거
- 은어/비속어 처리